In [1]:
import pickle
import re
import pandas as pd

# Data import & processing

In [2]:
df = pd.read_csv('data/clean_tweets.csv')
df3 = df.sort_values('likes_count', ascending=False)[:3]
df3

,id,tweet,created_at,user_id,name,username,retweets_count,replies_count,likes_count,quote_url
24,1519706866703638529,En breve empezamos la charla sobre RV y Psicol...,2022-04-28 17:55:26 CEST,3294953441,Jose Maria Torralba,jm_torralba_,1,0,15,NaN
93,1500124541137428490,Día para agradecer a @TheBridge_Tech y @Glovo_...,2022-03-05 16:02:16 CET,363153431,Pablo Oliete,pablooliete,0,0,13,NaN
62,1513848281801056258,"De los 21, hay 7 desarrolladoras de @Adalab_Di...",2022-04-12 13:55:30 CEST,1609031,Mánu Fosela,manufosela,3,3,12,NaN


DUPLICATES

In [3]:
data = df3.copy()

data.drop_duplicates(subset='tweet', inplace=True)

PUNCTUATION MARKS

In [4]:
data['tweet']

24    En breve empezamos la charla sobre RV y Psicol...
93    Día para agradecer a @TheBridge_Tech y @Glovo_...
62    De los 21, hay 7 desarrolladoras de @Adalab_Di...
Name: tweet, dtype: object

In [5]:
signs = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\¿)|(\@)|(\#)|(\_)|(\-)|(\/)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")

def signs_tweets(tweet):
    return signs.sub('', tweet.lower())

data['tweet'] = data['tweet'].apply(signs_tweets)
data['tweet'].head()

24    en breve empezamos la charla sobre rv y psicol...
93    día para agradecer a thebridgetech y glovoes l...
62    de los  hay  desarrolladoras de adalabdigital ...
Name: tweet, dtype: object

REMOVE LINKS

In [6]:
def remove_links(data):
    return " ".join(['{link}' if ('https') in word else word for word in data.split()])

data['tweet'] = data['tweet'].apply(remove_links)
data['tweet'].head()

24    en breve empezamos la charla sobre rv y psicol...
93    día para agradecer a thebridgetech y glovoes l...
62    de los hay desarrolladoras de adalabdigital de...
Name: tweet, dtype: object

STOPWORDS

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

spanish_stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /home/n0t10n/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def remove_stopwords(data):
    return " ".join([word for word in data.split() if word not in spanish_stopwords])

data['tweet'] = data['tweet'].apply(remove_stopwords)
data.head()

,id,tweet,created_at,user_id,name,username,retweets_count,replies_count,likes_count,quote_url
24,1519706866703638529,breve empezamos charla rv psicología ready the...,2022-04-28 17:55:26 CEST,3294953441,Jose Maria Torralba,jm_torralba_,1,0,15,NaN
93,1500124541137428490,día agradecer thebridgetech glovoes oportunida...,2022-03-05 16:02:16 CET,363153431,Pablo Oliete,pablooliete,0,0,13,NaN
62,1513848281801056258,desarrolladoras adalabdigital geeksacademyes c...,2022-04-12 13:55:30 CEST,1609031,Mánu Fosela,manufosela,3,3,12,NaN


STEMMER

In [9]:
from nltk.stem.snowball import SnowballStemmer

def spanish_stemmer(x):
    stemmer = SnowballStemmer('spanish')
    return " ".join([stemmer.stem(word) for word in x.split()])

data['tweet'] = data['tweet'].apply(spanish_stemmer)
data['tweet'].head()

24    brev empez charl rv psicolog ready thebridgete...
93    dia agradec thebridgetech glov oportun dad dos...
62    desarroll adalabdigital geeksacademy codenotch...
Name: tweet, dtype: object

In [30]:
pd.set_option('display.max_colwidth', None)

# Model

DATA

In [31]:
data['tweet']

24                                                                            brev empez charl rv psicolog ready thebridgetech {link}
93    dia agradec thebridgetech glov oportun dad dos cracks form codeorg program bloqu curs hor duracion brav 🙌 🔝🦸🏻‍♂️🦸🏻‍♀️🏭🇪🇸 {link}
62                         desarroll adalabdigital geeksacademy codenotchm socratech thebridgetech itacademy nebulov fpdaw autodidact
Name: tweet, dtype: object

In [32]:
df3['tweet']

24                                                                                                                                        En breve empezamos la charla sobre RV y Psicología! Todo ready! @TheBridge_Tech  https://t.co/lW3fvKhcGZ
93    Día para agradecer a @TheBridge_Tech y @Glovo_ES la oportunidad que han dado a estos dos cracks para formarse con @codeorg en programación por bloques en un curso de 48 horas de duración. Bravo 🙌  🔝🦸🏻‍♂️🦸🏻‍♀️🏭🇪🇸  https://t.co/ZTCrroHQTc
62                                         De los 21, hay 7 desarrolladoras de @Adalab_Digital, 4 de @4GeeksAcademyES, 2 de @CodenotchMAD, 1 de @socratech_ , 1 de @TheBridge_Tech, 1 de @ITAcademy, 1 de @nebulova_es y de FP-DAW y autodidactas.
Name: tweet, dtype: object

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,2))

text = pd.DataFrame(df3['tweet']).rename(columns={'tweet': 'content'})
text['clean_content'] = data['tweet']
text.head()

,content,clean_content
24,En breve empezamos la charla sobre RV y Psicología! Todo ready! @TheBridge_Tech https://t.co/lW3fvKhcGZ,brev empez charl rv psicolog ready thebridgetech {link}
93,Día para agradecer a @TheBridge_Tech y @Glovo_ES la oportunidad que han dado a estos dos cracks para formarse con @codeorg en programación por bloques en un curso de 48 horas de duración. Bravo 🙌 🔝🦸🏻‍♂️🦸🏻‍♀️🏭🇪🇸 https://t.co/ZTCrroHQTc,dia agradec thebridgetech glov oportun dad dos cracks form codeorg program bloqu curs hor duracion brav 🙌 🔝🦸🏻‍♂️🦸🏻‍♀️🏭🇪🇸 {link}
62,"De los 21, hay 7 desarrolladoras de @Adalab_Digital, 4 de @4GeeksAcademyES, 2 de @CodenotchMAD, 1 de @socratech_ , 1 de @TheBridge_Tech, 1 de @ITAcademy, 1 de @nebulova_es y de FP-DAW y autodidactas.",desarroll adalabdigital geeksacademy codenotchm socratech thebridgetech itacademy nebulov fpdaw autodidact


MODEL IMPORT

In [34]:
model = pickle.load(open('model/finished_model.model', 'rb'))

/home/n0t10n/Documents/Projects/Technical_test/.env/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/n0t10n/Documents/Projects/Technical_test/.env/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/n0t10n/Documents/Projects/Technical_test/.env/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Pipeline from

PREDICTIONS

In [35]:
predictions = model.predict(text['clean_content'])
predictions_proba = model.predict_proba(text['clean_content'])

text['Polarity_Pos'] = pd.Series(predictions_proba[0][0])
text['Polarity_Neg'] = pd.Series(predictions_proba[0][1])

text['Polarity'] = pd.Series(predictions)
text

,content,clean_content,Polarity_Pos,Polarity_Neg,Polarity
24,En breve empezamos la charla sobre RV y Psicología! Todo ready! @TheBridge_Tech https://t.co/lW3fvKhcGZ,brev empez charl rv psicolog ready thebridgetech {link},NaN,NaN,NaN
93,Día para agradecer a @TheBridge_Tech y @Glovo_ES la oportunidad que han dado a estos dos cracks para formarse con @codeorg en programación por bloques en un curso de 48 horas de duración. Bravo 🙌 🔝🦸🏻‍♂️🦸🏻‍♀️🏭🇪🇸 https://t.co/ZTCrroHQTc,dia agradec thebridgetech glov oportun dad dos cracks form codeorg program bloqu curs hor duracion brav 🙌 🔝🦸🏻‍♂️🦸🏻‍♀️🏭🇪🇸 {link},NaN,NaN,NaN
62,"De los 21, hay 7 desarrolladoras de @Adalab_Digital, 4 de @4GeeksAcademyES, 2 de @CodenotchMAD, 1 de @socratech_ , 1 de @TheBridge_Tech, 1 de @ITAcademy, 1 de @nebulova_es y de FP-DAW y autodidactas.",desarroll adalabdigital geeksacademy codenotchm socratech thebridgetech itacademy nebulov fpdaw autodidact,NaN,NaN,NaN
